### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

# to prevent figure 
# to show in the notebook
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

import plotting as plotting

### TODO: update this

In [3]:
date = "24-12-06"
y_none = True
pathfile = f"Pulling data/{date}/triple/"

max_100 = True

### From here; automatic run

In [4]:
root = f"/Users/michaelmoret/Library/CloudStorage/GoogleDrive-michael@externa.bio/.shortcut-targets-by-id/1BdUNsBjDh5Gee_76jCiKB1C_CwG0ercP/{pathfile}"

MODE = "triple"
dev = False

In [5]:
if not root.endswith("/"):
    root += "/"

sep_category = "$" # separates categories like date, experiments, etc
sep_count = "%" # separated the number of hairs by experiment

In [6]:
all_dfs = []
all_fns = []

for file in os.listdir(root):
    if file.endswith(".txt"):
        print("\n*****************************")
        print(file)
        df = plotting.get_df_from_file(root + file,
                                       skip=2)
        # clean the df
        if MODE in file:
            df = plotting.clean_triple(df, max_100=max_100)
            assert "triple" in file
        else:
            raise ValueError(f"Not a {MODE} experiment")

        splitted_name = file.split(sep_category)
        # get the experiments; i.e. not the date
        # not the single or triple etc
        splitted_name = splitted_name[2:-1]
        print("splitted_name: ", splitted_name)
        # add the experiment name in the dataframe
        all_names = []
        for entries in splitted_name:
            times_name = entries.split(sep_count)
            _times = int(times_name[0])
            _name = times_name[1]
            all_names += [_name] * _times
        print(f"len df: {len(df)}, len names: {len(all_names)}")
        df["Name"] = all_names
        # remove nan
        df = df.dropna()
        all_dfs.append(df)
        all_fns.append(file.replace(".txt", ""))


*****************************
20241206$EN_IHNY$25%bleached_ control_locx$25%positive 5_LocxEF$25%positive 1_LOcxEF$25%negative 2$25%NAC positive 3_Locx$25%NAC negative 4$25%LOcx negative 6$triple.txt
Missing records: []
splitted_name:  ['25%bleached_ control_locx', '25%positive 5_LocxEF', '25%positive 1_LOcxEF', '25%negative 2', '25%NAC positive 3_Locx', '25%NAC negative 4', '25%LOcx negative 6']
len df: 175, len names: 175


In [7]:
def master_plots(df, header, ymin, ymax, savepath, show=False, verbose=False):
    fig, ax, removed = plotting.create_boxplot(df, header, ymin, ymax)
    #save removed data
    removed.to_csv(f"{savepath}{header}_removed.csv", index=True)
    if verbose:
        print(f"{len(removed)} removed data points for {header}")
    # plot
    fig.savefig(f"{savepath}{header}.png")
    if show: 
        plt.show()
        plt.close()

In [8]:
len(all_dfs), len(all_fns)

(1, 1)

In [9]:
all_fns

['20241206$EN_IHNY$25%bleached_ control_locx$25%positive 5_LocxEF$25%positive 1_LOcxEF$25%negative 2$25%NAC positive 3_Locx$25%NAC negative 4$25%LOcx negative 6$triple']

In [10]:
for df, name in zip(all_dfs, all_fns):
    print("\n************************************")
    print(f"{name}\n")
    print(f"df length: {len(df)}")
    try:
        savepath = f"{root}{name}/"
        if dev:
            savepath = f"{root}dev/"
        os.makedirs(savepath, exist_ok=True)
        
        try:
            header = 'MEAN DIAMETER'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 20
                ymax = 120
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK STRESS'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 120
                ymax = 280
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'TOUGHNESS'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = None
                ymax = None
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)
        
        try:
            header = 'ELASTIC GRADIENT'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 0
                ymax =  140
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'ELASTIC EMOD'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 2.5
                ymax =  6.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        # scatter plots
        y_col = 'ELASTIC EMOD'
        x_col = 'MEAN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'ELASTIC EMOD'
        x_col = 'MIN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'ELASTIC EMOD'
        x_col = 'MAX DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'MEAN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'MIN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'MAX DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'RECORD'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

    except:
        print(f"\nERROR with {name}\n")
        continue

print("\n\nplotting DONE")


************************************
20241206$EN_IHNY$25%bleached_ control_locx$25%positive 5_LocxEF$25%positive 1_LOcxEF$25%negative 2$25%NAC positive 3_Locx$25%NAC negative 4$25%LOcx negative 6$triple

df length: 156

Creating boxplot for MEAN DIAMETER
Removed outliers for bleached_ control_locx : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for positive 5_LocxEF : 43    99.9
Name: MEAN DIAMETER, dtype: float64
Removed outliers for positive 1_LOcxEF : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for negative 2 : 97    96.8
Name: MEAN DIAMETER, dtype: float64
Removed outliers for NAC positive 3_Locx : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for NAC negative 4 : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for LOcx negative 6 : Series([], Name: MEAN DIAMETER, dtype: float64)
Stats. significance: negative 2 and NAC positive 3_Locx: p = 0.0429

Creating boxplot for BREAK STRESS
Removed outliers for blea